In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from variableUtils import *
import variableUtils
from Utils import *
# from ClassUtils import *
from pprint import pprint
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [ ]:
filepath = 'FullSpreadsheets\CAF+v0.1_September+29,+2024_20.45\CAF v0.1_September 29, 2024_20.45.csv'
folder, filename, ext = getFolderandFileName(filepath)
df = pd.read_csv(filepath)
# print(df[colComments])

In [ ]:
uniqueComments = df[colComments].unique()
uniqueComments2 = df['Further comments'].unique()
# remove nan
uniqueComments = uniqueComments[~pd.isnull(uniqueComments)]
uniqueComments2 = uniqueComments2[~pd.isnull(uniqueComments2)]
comments = uniqueComments.tolist() + uniqueComments2.tolist()
print(len(comments))
for comment in comments:
    print(comment)
    

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams
from collections import Counter
import string

# 1. Preprocess the text
def preprocess_text(text):
    # Convert to lower case
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize words
    tokens = nltk.word_tokenize(text)
    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return tokens

# 2. Extract phrases (n-grams)
def extract_phrases(comments, n=2):
    all_phrases = []
    for comment in comments:
        tokens = preprocess_text(comment)
        phrases = list(ngrams(tokens, n))
        all_phrases.extend(phrases)
    return all_phrases

# 3. Get most common phrases
def get_common_phrases(comments, n=2):
    phrases = extract_phrases(comments, n)
    phrase_counter = Counter(phrases)
    return phrase_counter.most_common(10)  # Get top 10 most common phrases

# Example usage:
for comment in comments[:10]:
    print(comment)
    tokens = preprocess_text(comment)
    print(tokens)



In [ ]:
# Example usage:
common_phrases = get_common_phrases(comments, 2)
print("Common Phrases:", common_phrases)

In [ ]:
from keybert import KeyBERT
# from keyphrase_vectorizers import KeyphraseCountVectorizer
# 1. Combine all comments into one large text corpus
corpus = ' '.join(comments)

# 2. Initialize KeyBERT model
kw_model = KeyBERT()

# 3. Extract key phrases from the entire corpus
key_phrases = kw_model.extract_keywords(corpus, keyphrase_ngram_range=(2, 5), top_n=20, stop_words='english')

print("Key Phrases:", key_phrases)

### Topic Modelling

In [ ]:
import gensim
from gensim import corpora
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string

# Sample data: list of comments
# documents = ["Comment one text goes here", "Comment two text goes here", ...]

# Step 1: Preprocessing
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free = " ".join([word for word in doc.lower().split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in comments]

# Step 2: Preparing Document-Term Matrix
dictionary = corpora.Dictionary(doc_clean)
corpus = [dictionary.doc2bow(text) for text in doc_clean]

# Step 3: LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=50)

# Step 4: Results
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)


In [ ]:
import spacy
import numpy as np
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
from sklearn.metrics import pairwise_distances_argmin_min

# Load pre-trained model
nlp = spacy.load('en_core_web_sm')
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Example comments
# comments = [
#     "I love the new update, it’s sleek and easy to use.",
#     "The new update is amazing, very user-friendly.",
#     "Not happy with the service, it was very slow.",
#     "The service was disappointing, took too long to respond.",
#     "Great job on the latest event, very well organized!",
#     "The event was fantastic, had a great time!"
# ]

# Step 1: Preprocess comments
def preprocess(text):
    doc = nlp(text.lower())
    result = ' '.join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])
    return result

preprocessed_comments = [preprocess(comment) for comment in comments]

# Step 2: Convert text to vectors
embeddings = sbert_model.encode(preprocessed_comments)




In [ ]:
# Clustering
num_clusters = 50
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(embeddings)
labels = kmeans.labels_

# Organize comments by clusters
clustered_comments = {i: [] for i in range(num_clusters)}
for comment, label in zip(comments, labels):
    clustered_comments[label].append(comment)

# Print all comments for each cluster
for cluster, cluster_comments in clustered_comments.items():
    print(f"Cluster {cluster + 1}:")
    for comment in cluster_comments:
        print(f" - {comment}")
    print()  # Add an empty line for better separation

# save the clusters and the comments in a file
with open(f'{folder}/clusters.txt', 'w') as f:
    for cluster, cluster_comments in clustered_comments.items():
        f.write(f"Cluster {cluster + 1}:\n")
        for comment in cluster_comments:
            f.write(f" - {comment}\n")
        f.write("\n\n")

In [ ]:
def calc(n, tot):
    return (tot - n) / tot * 100
tot = [7, 4, 3, 5]

x=[1, 0, 0, 1]
# for i in range(4):
#     print(calc(x[i], tot[i]))
xlist = [[1, 0, 0, 1], [0, 0, 0, 0], [2, 0, 0, 2], [0, 0, 0, 1], [1, 0, 0, 1], [0, 0, 0, 1], [1, 0, 0, 1], [3, 0, 0, 0]]
for x in xlist:
    print(calc(x[0], 7), calc(x[1], 4), calc(x[2], 3), calc(x[3], 5))

## To get words that occur togther and are interesting

In [ ]:
import nltk
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

# change this to read in your data
finder = BigramCollocationFinder.from_words(
    nltk.corpus.genesis.words('english-web.txt'))

# only bigrams that appear 3+ times
finder.apply_freq_filter(3)

# return the 10 n-grams with the highest PMI
finder.nbest(bigram_measures.pmi, 10)